In [ ]:

!pip install scikit-image
!pip install scikit-learn
!pip install opencv-python
!pip install opencv-contrib-python
!pip install numpy
!pip install matplotlib
!pip install ultralytics opencv-python
!pip install scikit-learn joblib tqdm


In [ ]:
import os
import glob
import cv2
import torch
import numpy as np
import pandas as pd
import joblib
import itertools
from matplotlib import pyplot as plt
from ultralytics import YOLO
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.signal import convolve2d
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
from tqdm.notebook import tqdm

# 불필요한 경고 메시지는 무시합니다.
warnings.filterwarnings('ignore')

#### 데이터셋 다운로드
학습 및 평가에 사용할 recaptcha-dataset을 GitHub에서 복제하여 가져옵니다.

In [ ]:
!git clone https://github.com/hbcbh1999/recaptcha-dataset

#### 2.1. 이미지 전처리 유틸리티 함수
모든 피처 추출 단계 이전에 공통으로 필요한 이미지 변환(그레이스케일, CLAHE, HSV 변환)을 수행하는 헬퍼 함수를 정의합니다.

In [ ]:
def prepare_image_channels(bgr_image):
    gray = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)를 적용하여 이미지 대비를 향상시킵니다.
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray_clahe = clahe.apply(gray)

    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)

    return {"gray": gray, "gray_clahe": gray_clahe, "hsv": hsv}

#### 2.2. 객체 탐지 기반 이미지 크롭 클래스
YOLOv8 모델을 사용하여 이미지에서 분석과 관련 없는 배경(clutter)을 제거하고, 오직 핵심 객체(Object of Interest)에만 집중하여 피처의 품질을 극대화합니다.

In [ ]:
class YoloObjectCropper:
    """YOLO 모델을 사용해 이미지에서 특정 클래스의 객체를 탐지하고 잘라냅니다."""
    def __init__(self, model_name='yolov8n.pt', margin_ratio=0.1):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_name).to(self.device)
        self.margin_ratio = margin_ratio

        # 데이터셋의 클래스 이름과 YOLO 모델의 클래스 이름을 매핑합니다.
        # 값이 None이면 해당 클래스는 객체 탐지 없이 원본 이미지를 그대로 사용합니다.
        self.class_map = {
            'Bicycle': 'bicycle', 'Bridge': None, 'Bus': 'bus', 'Car': 'car',
            'Chimney': None, 'Crosswalk': None, 'Hydrant': 'fire hydrant',
            'Motorcycle': 'motorcycle', 'Palm': 'potted plant', 'Traffic Light': 'traffic light'
        }

    def crop_object(self, image_path, expected_class_name):
        try:
            image = cv2.imread(image_path)
            if image is None: return None
            h, w, _ = image.shape
        except Exception:
            return None

        yolo_class_name = self.class_map.get(expected_class_name)
        # YOLO 클래스 맵에 없는 경우 원본 이미지 반환
        if yolo_class_name is None:
            return image

        results = self.model(image, verbose=False)
        best_box, max_conf = None, 0.0

        # 탐지된 여러 객체 중, 신뢰도(confidence)가 가장 높은 객체를 선택
        for box in results[0].boxes:
            d_cls_name = self.model.names[int(box.cls[0])]
            if d_cls_name == yolo_class_name and float(box.conf[0]) > max_conf:
                max_conf, best_box = float(box.conf[0]), box.xyxy[0].cpu().numpy()

        if best_box is not None:
            x1, y1, x2, y2 = map(int, best_box)
            # 객체 주변에 여백(margin)을 추가하여 정보 손실을 최소화
            mw, mh = int((x2 - x1) * self.margin_ratio), int((y2 - y1) * self.margin_ratio)
            return image[max(0, y1 - mh):min(h, y2 + mh), max(0, x1 - mw):min(w, x2 + mw)]

        return image # 객체를 찾지 못하면 원본 이미지 반환

#### 2.3. 종합적인 이미지 피처 추출 클래스
##### 2.3.1. Simultaneous extraction of individual features
- 키포인트 기반: AKAZE, SIFT
- 형태 기반: HOG
- 텍스처 기반: GLCM 통계량, Law's Texture Energy
- 색상 기반: HSV 색 공간에서의 Color Histogram

##### 2.3.2. Concatenation
모든 개별 Feature vector 를 하나의 긴 vector로 연결합니다.

In [ ]:
class ComprehensiveFeatureExtractor:
    """SIFT, AKAZE, HOG, Color, LBP, GLCM, Law's Texture 등 다양한 피처를 종합적으로 추출합니다."""
    def __init__(self, hog_win_size=(64, 128), lbp_radius=3, lbp_points=24):
        # SIFT, AKAZE는 디스크립터 크기가 고정되어 있으므로 미리 정의합니다.
        self.sift = cv2.SIFT_create()
        self.sift_desc_size = 128
        self.akaze = cv2.AKAZE_create()
        self.akaze_desc_size = 61

        self.hog_win_size = hog_win_size
        self.hog = cv2.HOGDescriptor(hog_win_size, (16,16), (8,8), (8,8), 9)
        self.lbp_radius, self.lbp_points = lbp_radius, lbp_points

    def _extract_keypoint_descriptors(self, gray_image, extractor, desc_size):
        """키포인트 디스크립터를 추출하고, 이미지 전체를 대표하는 하나의 평균 벡터로 변환합니다."""
        _, descs = extractor.detectAndCompute(gray_image, None)
        # 키포인트가 없는 경우 0벡터 반환
        if descs is None:
            return np.zeros(desc_size)
        return np.mean(descs, axis=0)

    def _extract_hog(self, gray):
        resized_gray = cv2.resize(gray, self.hog_win_size)
        return self.hog.compute(resized_gray).flatten()

    def _extract_color_histogram(self, hsv, bins=(8,12,3)):
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256]).flatten()
        # L1 정규화
        return hist / np.sum(hist) if np.sum(hist) > 0 else hist

    def _extract_lbp(self, gray):
        lbp = local_binary_pattern(gray, self.lbp_points, self.lbp_radius, 'uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, self.lbp_points + 3), range=(0, self.lbp_points + 2))
        return hist.astype("f") / hist.sum() if hist.sum() > 0 else hist

    def _extract_glcm(self, gray):
        glcm = graycomatrix(gray, [1, 3, 5], [0, np.pi/4, np.pi/2, 3*np.pi/4], 256, symmetric=True, normed=True)
        props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        return np.hstack([graycoprops(glcm, p_name).flatten() for p_name in props])

    def _extract_laws_texture(self, gray):
        """Law's Texture Energy 마스크를 적용하여 텍스처 에너지를 측정합니다."""
        v = np.array
        # 1D 필터 커널 정의: L(Level), E(Edge), S(Spot), W(Wave), R(Ripple)
        L, E, S, W, R = v([1,4,6,4,1]), v([-1,-2,0,2,1]), v([-1,0,2,0,-1]), v([-1,2,0,-2,1]), v([1,-4,6,-4,1])
        kernels = {'L5':L,'E5':E,'S5':S,'W5':W,'R5':R}
        # 1D 커널들을 외적(outer product)하여 2D 컨볼루션 마스크 9개를 생성
        masks = [np.outer(v1,v2) for i,(n1,v1) in enumerate(kernels.items()) for n2,v2 in list(kernels.items())[i:]][:9]

        norm_gray = gray.astype(np.float32) - convolve2d(gray, np.ones((5,5))/25, 'same', 'symm')
        # 각 마스크를 적용한 후 에너지(절대값의 평균)를 계산하여 피처 벡터 생성
        return v([np.mean(np.abs(convolve2d(norm_gray, m, 'same', 'symm'))) for m in masks])

    def extract_all(self, preprocessed_images):
        """정의된 모든 피처 추출 기법을 순차적으로 적용하고 하나의 벡터로 결합합니다."""
        v_sift = self._extract_keypoint_descriptors(preprocessed_images["gray_clahe"], self.sift, self.sift_desc_size)
        v_akaze = self._extract_keypoint_descriptors(preprocessed_images["gray_clahe"], self.akaze, self.akaze_desc_size)
        v_hog = self._extract_hog(preprocessed_images["gray_clahe"])
        v_color = self._extract_color_histogram(preprocessed_images["hsv"])
        v_lbp = self._extract_lbp(preprocessed_images["gray"])
        v_glcm = self._extract_glcm(preprocessed_images["gray"])
        v_law = self._extract_laws_texture(preprocessed_images["gray"])

        return np.hstack([v_sift, v_akaze, v_hog, v_color, v_lbp, v_glcm, v_law])

#### 2.4. 소프트 보팅(Soft Voting) 앙상블 분류기
여러 개의 KNN 모델(각각 다른 피처셋으로 학습)의 예측 확률을 종합하여 최종 클래스를 결정하는 앙상블 모델입니다.

In [1]:
class KnnSoftVotingClassifier:
    """여러 KNN 모델의 예측 확률을 평균 내어 최종 예측을 수행하는 Soft Voting 앙상블 모델입니다."""
    def __init__(self, n_neighbors=10):
        self.feature_names = []
        self.models = {}
        self.classes_ = None
        self.n_neighbors = n_neighbors

    def fit(self, feature_sets, labels):
        """피처셋별로 별도의 KNN 모델을 학습시킵니다."""
        self.feature_names = list(feature_sets.keys())
        self.classes_ = np.unique(labels)

        for name in self.feature_names:
            # `weights='distance'`는 예측 시 가까운 이웃에 더 큰 가중치를 부여합니다.
            knn = KNeighborsClassifier(n_neighbors=self.n_neighbors, weights='distance', n_jobs=-1)
            knn.fit(feature_sets[name], labels)
            self.models[name] = knn
        return self

    def predict_proba(self, query_features_dict):
        """모든 개별 모델의 예측 확률을 합산하고 정규화합니다."""
        num_samples = list(query_features_dict.values())[0].shape[0]
        total_probas = np.zeros((num_samples, len(self.classes_)))

        for name in self.feature_names:
            model = self.models[name]
            probas = model.predict_proba(query_features_dict[name])

            # 모델마다 클래스 순서가 다를 수 있으므로, 전체 클래스 배열 기준으로 확률을 더함
            for i, cls in enumerate(model.classes_):
                class_idx = np.where(self.classes_ == cls)[0][0]
                total_probas[:, class_idx] += probas[:, i]

        # 모든 확률의 합이 1이 되도록 정규화
        row_sums = total_probas.sum(axis=1, keepdims=True)
        safe_row_sums = np.where(row_sums == 0, 1, row_sums)
        return total_probas / safe_row_sums

    def predict(self, query_features_dict):
        """가장 높은 최종 확률을 가진 클래스를 예측 결과로 반환합니다."""
        final_probas = self.predict_proba(query_features_dict)
        best_class_indices = np.argmax(final_probas, axis=1)
        return self.classes_[best_class_indices]

#### 3.1. 피처 데이터베이스 구축
전체 이미지 데이터셋에 대해 `YoloObjectCropper`와 `ComprehensiveFeatureExtractor`를 순차적으로 적용하여, 모든 이미지로부터 고차원 피처 벡터를 추출하고 파일로 저장합니다.

In [ ]:
def build_feature_database(dataset_path='./recaptcha-dataset/Large'):
    """데이터셋의 모든 이미지에서 피처를 추출하여 원본 피처 데이터베이스를 구축합니다."""

    cropper = YoloObjectCropper()
    feature_extractor = ComprehensiveFeatureExtractor()
    target_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk',
                     'Hydrant', 'Motorcycle', 'Palm', 'Traffic Light']

    image_paths = []
    for label in target_labels:
        class_dir = os.path.join(dataset_path, label)
        if os.path.isdir(class_dir):
            image_paths.extend(glob.glob(os.path.join(class_dir, '*.png')))

    if not image_paths:
        print(f"오류: '{dataset_path}' 경로에서 타겟 클래스 이미지를 찾을 수 없습니다.")
        return

    all_raw_features, all_labels = [], []
    print(f"총 {len(image_paths)}개의 이미지에 대해 피처 추출을 시작합니다.")

    # 첫 이미지의 피처 길이를 기준으로 모든 피처의 길이를 통일
    expected_len = -1
    for img_path in tqdm(image_paths, desc="피처 데이터베이스 구축 중"):
        try:
            label = os.path.basename(os.path.dirname(img_path))

            # 1. 객체 영역 크롭
            cropped_image = cropper.crop_object(img_path, label)
            if cropped_image is None or cropped_image.shape[0] < 32 or cropped_image.shape[1] < 32:
                continue

            # 2. 이미지 채널 준비 및 피처 추출
            preprocessed_images = prepare_image_channels(cropped_image)
            raw_feature_vector = feature_extractor.extract_all(preprocessed_images)

            if expected_len == -1:
                expected_len = len(raw_feature_vector)
            if len(raw_feature_vector) != expected_len:
                # 피처 벡터 길이가 다른 경우, 해당 이미지는 건너뜀
                tqdm.write(f"CRITICAL ERROR: {img_path} 벡터 길이 불일치. 건너뜁니다.")
                continue

            all_raw_features.append(raw_feature_vector)
            all_labels.append(label)
        except Exception as e:
            tqdm.write(f"Error: {img_path} 처리 중 예외 발생 - {e}")

    X_raw, y = np.array(all_raw_features), np.array(all_labels)
    if X_raw.size == 0:
        print("오류: 피처 추출에 성공한 이미지가 없습니다.")
        return

    print(f"피처 추출 완료. 추출된 고차원 피처의 형태: {X_raw.shape}")

    # RandomForest를 사용한 피처 중요도 계산 및 상위 피처 선택
    # 여기서는 모든 피처를 그대로 사용하지만, 중요도 기반 선택 로직을 추가할 수 있습니다.
    selector = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1).fit(X_raw, y)

    # 결과 저장
    np.save('all_raw_features.npy', X_raw)
    np.save('all_labels.npy', y)
    joblib.dump(selector, 'feature_importance_model.pkl') # 피처 중요도 모델 저장
    print("피처 데이터베이스 및 중요도 모델 저장이 완료되었습니다.")

# 함수 실행
build_feature_database()

In [ ]:
def evaluate_final_model(test_dataset_path='./recaptcha-dataset/Large/'):
    """저장된 최종 모델들을 불러와 테스트 데이터에 대한 성능을 평가합니다."""
    try:
        final_ensemble_model = joblib.load('final_ensemble_model.pkl')
        final_pca_models = joblib.load('ensemble_pca_models.pkl')
    except FileNotFoundError as e:
        print(f"오류: 모델 파일을 찾을 수 없습니다 - {e}"); return

    # 피처 그룹별 인덱스 정보 (튜닝 단계와 동일하게 사용)
    feature_indices = {
        'keypoint': (0, 189), 'shape': (189, 4000), # shape, texture, color 인덱스 재정의 필요
        'texture': (4000, 4000+26+60+9),
        'color': (4000+26+60+9, None)
    }

    target_labels = final_ensemble_model.classes_
    query_image_paths = []
    for label in target_labels:
        image_dir = os.path.join(test_dataset_path, label)
        if os.path.isdir(image_dir):
            # 각 클래스별로 10개씩의 이미지를 테스트셋으로 사용
            query_image_paths.extend([os.path.join(image_dir, name) for name in sorted(os.listdir(image_dir))[10:20]])

    task1_results, true_labels = [], []
    cropper = YoloObjectCropper()
    feature_extractor = ComprehensiveFeatureExtractor()

    print(f"최종 앙상블 모델 평가: 총 {len(query_image_paths)}개 쿼리 이미지 평가 시작...")
    for img_path in tqdm(query_image_paths, desc="최종 모델 평가 진행 중"):
        true_label = os.path.basename(os.path.dirname(img_path))

        cropped_image = cropper.crop_object(img_path, true_label)
        if cropped_image is None: continue

        preprocessed_images = prepare_image_channels(cropped_image)
        # 평가 시에는 피처 인덱스를 다시 계산해야 하므로, 임시로 전체 피처를 사용합니다.
        # 실제로는 build_feature_database에서 사용한 피처와 동일한 shape이어야 함
        query_raw_feature = feature_extractor.extract_all(preprocessed_images)

        # 쿼리 피처셋 분리 및 PCA 변환
        query_features_dict = {}
        # NOTE: 이 부분은 저장된 X_raw의 shape을 기준으로 인덱스를 재계산해야 가장 정확합니다.
        # 아래는 예시 인덱스입니다.
        X_raw_shape_1 = 4352 # build_feature_database의 결과 shape
        feature_indices = {
            'keypoint': (0, 189), 'shape': (189, 189 + 3780),
            'texture': (189 + 3780, 189 + 3780 + 26 + 60 + 9),
            'color': (189 + 3780 + 26 + 60 + 9, X_raw_shape_1)
        }

        for name, (start, end) in feature_indices.items():
            feature_subset = query_raw_feature[start:end]
            feature_2d = feature_subset.reshape(1, -1)
            if name in final_pca_models:
                query_features_dict[name] = final_pca_models[name].transform(feature_2d)
            else:
                query_features_dict[name] = feature_2d

        final_prediction = final_ensemble_model.predict(query_features_dict)[0]
        task1_results.append([os.path.basename(img_path), final_prediction])
        true_labels.append(true_label)

    task1_predictions = [row[1] for row in task1_results]
    print("\n--- 최종 모델(Ultimate Ensemble) 성능 평가 보고서 ---")
    print(classification_report(true_labels, task1_predictions, labels=target_labels))

    pd.DataFrame(task1_results, columns=['Image', 'Predicted_Label']).to_csv('c1_t1_a1_predictions.csv', index=False)
    print("평가 결과가 'c1_t1_a1_predictions.csv' 파일로 저장되었습니다.")

# 함수 실행
evaluate_final_model()